In [1]:
import gurobipy as gp
from gurobipy import Model, GRB

In [2]:
Model()

Set parameter LicenseID to value 2599974


<gurobi.Model Continuous instance Unnamed: 0 constrs, 0 vars, Parameter changes: Username=(user-defined), LicenseID=2599974>

In [4]:
from gurobipy import Model, GRB, quicksum

# Data inputs (example values)
grids = ["Thoothukudi", "Chennai", "Coimbatore"]
demand = {"Thoothukudi": 500, "Chennai": 800, "Coimbatore": 600}  # in tonnes/day
wind_capacity = {"Thoothukudi": 1000, "Chennai": 300, "Coimbatore": 400}  # in tonnes/day
risk_score = {"Thoothukudi": 2, "Chennai": 5, "Coimbatore": 3}  # Safety risk score
facility_cost = {"Thoothukudi": 1e8, "Chennai": 1.5e8, "Coimbatore": 1.2e8}  # INR
operating_cost = {"Thoothukudi": 2e6, "Chennai": 3e6, "Coimbatore": 2.5e6}  # INR/day
transport_cost_per_km = 7  # INR/kg/100 km
distances = {
    ("Thoothukudi", "Chennai"): 600,
    ("Thoothukudi", "Coimbatore"): 400,
    ("Chennai", "Coimbatore"): 500,
}  # in km

# Model setup
model = Model("Hydrogen_Supply_Chain")

# Decision variables
P = model.addVars(grids, name="Production", vtype=GRB.CONTINUOUS)  # Production
Q = model.addVars(grids, grids, name="Transport", vtype=GRB.CONTINUOUS)  # Transport flow
F = model.addVars(grids, name="Facility", vtype=GRB.BINARY)  # Facility presence

# Objective: Minimize cost + safety, maximize productivity
model.setObjective(
    quicksum(facility_cost[g] * F[g] for g in grids)  # Facility costs
    + quicksum(operating_cost[g] * P[g] for g in grids)  # Operating costs
    + quicksum(Q[g1, g2] * transport_cost_per_km * distances[(g1, g2)] / 100 for g1, g2 in distances)  # Transport costs
    + quicksum(P[g] * risk_score[g] for g in grids),  # Safety risk
    GRB.MINIMIZE,
)

# Constraints
# 1. Demand satisfaction
for d in grids:
    model.addConstr(quicksum(Q[g, d] for g in grids if g != d) >= demand[d], f"Demand_{d}")

# 2. Production capacity
for g in grids:
    model.addConstr(P[g] <= wind_capacity[g] * F[g], f"ProductionCap_{g}")

# 3. Facility count
model.addConstr(quicksum(F[g] for g in grids) >= 2, "FacilityCount")

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Production Plan:")
    for g in grids:
        print(f"{g}: {P[g].x:.2f} tonnes/day")
    print("\nFacility Plan:")
    for g in grids:
        print(f"{g}: {'Active' if F[g].x > 0.5 else 'Inactive'}")
    print("\nTransport Plan:")
    for g1, g2 in distances:
        if Q[g1, g2].x > 0:
            print(f"From {g1} to {g2}: {Q[g1, g2].x:.2f} tonnes/day")
    print(f"\nTotal Cost: {model.objVal:.2f} INR/day")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 15 columns and 15 nonzeros
Model fingerprint: 0x08a9af9d
Variable types: 12 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+01, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+02]
Found heuristic solution: objective 2.700210e+08
Presolve removed 7 rows and 15 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 2.20017e+08 2.70021e+08 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200168000000e+08, best bound 2.200168000000e+08, gap 0.0000%
Optimal sol

In [5]:
# Create the model
model = Model("Hydrogen Supply Chain Optimization")

# Parameters (example values; replace with real-world data)
demand = 13393260  # kg/day
C_cap_p, C_cap_s, C_cap_t = 47310, 8022, 150  # Capital cost coefficients ($)
C_op_p, C_op_s, C_op_t = 2116, 126, 1  # Operating cost coefficients ($/day)
emission_p, emission_s, emission_t = 135.27, 70.33, 0.261  # Emissions (kg CO2)
risk_p, risk_s, risk_t = 580, 5155, 4557  # Relative risk
min_facilities = 2
max_facilities = 4

# Decision variables
x_p = model.addVar(vtype=GRB.INTEGER, lb=min_facilities, ub=max_facilities, name="ProductionFacilities")
x_s = model.addVar(vtype=GRB.INTEGER, lb=1, ub=10, name="StorageFacilities")
x_t = model.addVar(vtype=GRB.INTEGER, lb=1, ub=10, name="TransportUnits")

# Objective 1: Minimize Cost
cost = (C_cap_p * x_p + C_cap_s * x_s + C_cap_t * x_t +
        C_op_p * x_p + C_op_s * x_s + C_op_t * x_t)

# Objective 2: Minimize Global Warming Potential (GWP)
gwp = emission_p * x_p + emission_s * x_s + emission_t * x_t

# Objective 3: Minimize Risk
risk = risk_p * x_p + risk_s * x_s + risk_t * x_t

# Combine objectives using weighted sum approach
model.setObjective(0.5 * cost + 0.3 * gwp + 0.2 * risk, GRB.MINIMIZE)

# Constraints
model.addConstr(x_p * 500000 >= demand, "DemandFulfillment")  # Production capacity meets demand
model.addConstr(x_s >= x_p, "StorageConstraint")  # Storage facilities at least equal to production facilities

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal Solution:")
    print(f"Production Facilities: {x_p.x}")
    print(f"Storage Facilities: {x_s.x}")
    print(f"Transport Units: {x_t.x}")
    print(f"Total Cost: {cost.getValue()}")
    print(f"Total GWP: {gwp.getValue()}")
    print(f"Total Risk: {risk.getValue()}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x9fa90867
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+07, 1e+07]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


In [6]:
# Initialize model
model = Model("Hydrogen_Supply_Chain_Optimization")

# Parameters (example data based on typical supply chain figures)
cost_production_facilities = [43000, 57000, 90000]  # Cost of 2, 3, 4 facilities respectively (in M$)
num_production_facilities = [2, 3, 4]
cost_storage = 5000  # Fixed cost for storage facilities (in M$)
cost_transportation = 1000  # Cost per transportation unit (in M$)
demand = 133392360  # Annual hydrogen demand (in kg)
efficiency = 0.65  # Efficiency of hydrogen production per unit power input

# Decision Variables
x = model.addVars(len(num_production_facilities), vtype=GRB.BINARY, name="x")  # Whether to use each facility setup
y_storage = model.addVar(vtype=GRB.INTEGER, name="y_storage")  # Number of storage facilities
z_transport = model.addVar(vtype=GRB.INTEGER, name="z_transport")  # Number of transportation units

# Objective Function: Minimize Total Cost
model.setObjective(
    quicksum(x[i] * cost_production_facilities[i] for i in range(len(num_production_facilities))) +
    cost_storage * y_storage +
    cost_transportation * z_transport,
    GRB.MINIMIZE
)

# Constraints
# Constraint: Minimum demand must be met
model.addConstr(
    quicksum(x[i] * num_production_facilities[i] * efficiency * demand for i in range(len(num_production_facilities))) >= demand,
    "Demand_Constraint"
)

# Constraint: Safety factor (simplified)
safety_factor = 0.9  # Assume 90% safety reliability
model.addConstr(
    quicksum(x[i] for i in range(len(num_production_facilities))) * safety_factor >= 2,
    "Safety_Constraint"
)

# Solve the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}")
    print(f"Total cost: {model.objVal} M$")
else:
    print("No optimal solution found")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x76de6ba8
Variable types: 0 continuous, 5 integer (3 binary)
Coefficient statistics:
  Matrix range     [9e-01, 3e+08]
  Objective range  [1e+03, 9e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 190000.00000
Presolve removed 2 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 190000 

Optimal solution found (tolerance 1.00e-04)
Be

In [8]:
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

# Parameters (Replace with realistic data for Tamil Nadu, 2024)
demand = 13393260  # kg/day (daily hydrogen demand)
C_cap_p, C_cap_s, C_cap_t = 47310, 8022, 150  # Capital costs in $
C_op_p, C_op_s, C_op_t = 2116, 126, 1  # Operating costs in $/day
emission_p, emission_s, emission_t = 135.27, 70.33, 0.261  # Emissions (kg CO2-equivalent)
risk_p, risk_s, risk_t = 580, 5155, 4557  # Relative risk values
min_facilities = 2
max_facilities = 4

# Create the Gurobi model
model = Model("HydrogenSupplyOptimization")

# Decision variables
x_p = model.addVar(vtype=GRB.INTEGER, lb=min_facilities, ub=max_facilities, name="ProductionFacilities")
x_s = model.addVar(vtype=GRB.INTEGER, lb=1, ub=10, name="StorageFacilities")
x_t = model.addVar(vtype=GRB.INTEGER, lb=1, ub=10, name="TransportUnits")

# Objective 1: Minimize Cost
cost = (C_cap_p * x_p + C_cap_s * x_s + C_cap_t * x_t +
        C_op_p * x_p + C_op_s * x_s + C_op_t * x_t)

# Objective 2: Minimize Global Warming Potential (GWP)
gwp = emission_p * x_p + emission_s * x_s + emission_t * x_t

# Objective 3: Minimize Risk
risk = risk_p * x_p + risk_s * x_s + risk_t * x_t

# Combine objectives using a weighted sum approach
# Weights chosen: Cost (0.5), GWP (0.3), Risk (0.2)
model.setObjective(0.5 * cost + 0.3 * gwp + 0.2 * risk, GRB.MINIMIZE)

# Constraints
model.addConstr(x_p * 500000 >= demand, "DemandFulfillment")  # Production must meet demand
model.addConstr(x_s >= x_p, "StorageConstraint")  # Storage must be at least equal to production

# Optimize the model
model.optimize()

# Check and display the results
if model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Production Facilities: {int(x_p.x)}")
    print(f"Storage Facilities: {int(x_s.x)}")
    print(f"Transport Units: {int(x_t.x)}")
    print(f"Total Cost: ${cost.getValue():,.2f}")
    print(f"Total GWP: {gwp.getValue():,.2f} kg CO2-equivalent")
    print(f"Total Risk: {risk.getValue():,.2f} units")
else:
    print("No optimal solution found.")

# Scenario analysis: Evaluate the impact of 2, 3, and 4 production facilities
for facilities in range(2, 5):
    x_p.lb = facilities
    x_p.ub = facilities
    model.optimize()
    if model.status == GRB.OPTIMAL:
        print(f"\nScenario with {facilities} Production Facilities:")
        print(f"Storage Facilities: {int(x_s.x)}")
        print(f"Transport Units: {int(x_t.x)}")
        print(f"Total Cost: ${cost.getValue():,.2f}")
        print(f"Total GWP: {gwp.getValue():,.2f} kg CO2-equivalent")
        print(f"Total Risk: {risk.getValue():,.2f} units")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x9fa90867
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+07, 1e+07]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX

In [11]:
# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 13690000

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Create optimization model
model = Model("Hydrogen Optimization")

# Decision variables
num_production_facilities = model.addVar(vtype=GRB.INTEGER, lb=min_facilities, ub=max_facilities, name="num_production_facilities")
storage_capacity = model.addVar(vtype=GRB.CONTINUOUS, name="storage_capacity")
transport_capacity = model.addVar(vtype=GRB.CONTINUOUS, name="transport_capacity")

# Objective coefficients
cost = (capital_cost_production * num_production_facilities +
        capital_cost_storage * storage_capacity +
        capital_cost_transport * transport_capacity +
        operating_cost_production * num_production_facilities * 365 +
        operating_cost_storage * storage_capacity * 365 +
        operating_cost_transport * transport_capacity * 365)

gwp = (GWP_production * num_production_facilities +
       GWP_storage * storage_capacity +
       GWP_transport * transport_capacity)

risk = (risk_production * num_production_facilities +
        risk_storage * storage_capacity +
        risk_transport * transport_capacity)

# Objective: Minimize cost
model.setObjective(cost, GRB.MINIMIZE)

# Constraints
model.addConstr(storage_capacity >= demand, "Storage Capacity Constraint")
model.addConstr(transport_capacity >= demand * 10, "Transport Capacity Constraint")  # Assume 10 km distance per tonne

# Optimize
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Optimal number of production facilities: {num_production_facilities.x}")
    print(f"Optimal storage capacity: {storage_capacity.x} tonnes")
    print(f"Optimal transport capacity: {transport_capacity.x} tonne-km")
    print(f"Total Cost: {cost.getValue()} crore ₹")
    print(f"Total GWP: {gwp.getValue()} 10^3 t CO2-equiv per day")
    print(f"Total Risk: {risk.getValue()} units")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 2 nonzeros
Model fingerprint: 0xc8dcb35f
Variable types: 2 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+02]
  Bounds range     [2e+00, 4e+00]
  RHS range        [1e+07, 1e+08]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.54561e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.545613712500e+07, best bound 6.545613712500e+07, gap 0.0000%
Optimal number of production facilities: 2.0
Optimal storage capacity: 13690

In [15]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

total_cost = capital_cost + operating_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Number of Facilities: {num_facilities.x}")
    print(f"Optimal Number of Storage Units: {num_storage_units.x}")
    print(f"Optimal Number of Transport Units: {num_transport_units.x}")
    print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
    print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
    print(f"Total Risk (unitless): {total_risk.getValue()}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x7253f242
Variable types: 2 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  Bounds range     [2e+00, 4e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 1902.7755000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1902.78 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.902775500000e+03, best bound 1.902775500000e+03, gap 0.0000%
Optimal Number of Facilities: 3

In [16]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# New variable for extra production capacity
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

# Display results for all the configurations (2, 3, and 4 facilities)
def display_results(num_facilities_value):
    # Update the number of facilities
    model.getVarByName("num_facilities").start = num_facilities_value
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xd4ae4f1e
Model has 1 quadratic objective term
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  QObjective range [7e+02, 7e+02]
  Bounds range     [2e+00, 4e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 2841243.9300
Presolve removed 2 rows and 2 columns
Presolve time: 0.05s
Presolved: 3 rows, 4 columns, 6 nonzeros
Presolved model has 1 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving 

In [18]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# New variable for extra production capacity
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

# Display results for all the configurations (2, 3, and 4 facilities)
def display_results(num_facilities_value):
    # Update the number of facilities
    model.getVarByName("num_facilities").start = num_facilities_value
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xd4ae4f1e
Model has 1 quadratic objective term
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  QObjective range [7e+02, 7e+02]
  Bounds range     [2e+00, 4e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 2841243.9300
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 6 nonzeros
Presolved model has 1 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving 

In [19]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# New variable for extra production capacity
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Binary variable to decide if more facilities need to be added
add_facilities = model.addVar(vtype=GRB.BINARY, name="add_facilities")

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Add constraint to decide if extra facilities are needed
model.addConstr(num_facilities >= min_facilities + add_facilities, "Add facilities constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Optimize
model.optimize()

# Display results for all the configurations (2, 3, and 4 facilities)
def display_results(num_facilities_value):
    # Update the number of facilities
    model.getVarByName("num_facilities").start = num_facilities_value
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x3691fc8b
Model has 1 quadratic objective term
Variable types: 3 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  QObjective range [7e+02, 7e+02]
  Bounds range     [1e+00, 4e+00]
  RHS range        [2e+00, 1e+03]
Found heuristic solution: objective 2841243.9300
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 6 nonzeros
Presolved model has 1 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving 

In [20]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# New variable for extra production capacity (per existing facility)
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Binary variable to decide if more facilities need to be added
add_facilities = model.addVar(vtype=GRB.BINARY, name="add_facilities")

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Add constraint to decide if extra facilities are needed
model.addConstr(num_facilities >= min_facilities + add_facilities, "Add facilities constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Optimize for multiple configurations (2, 3, and 4 facilities)
def display_results(num_facilities_value):
    # Update the number of facilities
    model.getVarByName("num_facilities").start = num_facilities_value
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


GurobiError: No variable names available to index

In [25]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# New variable for extra production capacity (per existing facility)
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Binary variable to decide if more facilities need to be added
add_facilities = model.addVar(vtype=GRB.BINARY, name="add_facilities")

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Add production capacity constraint: total production should be less than or equal to the total production capacity of the selected number of facilities
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Add constraint to decide if extra facilities are needed
model.addConstr(num_facilities >= min_facilities + add_facilities, "Add facilities constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 6 nonzeros
Model fingerprint: 0xbe58aa00
Model has 1 quadratic objective term
Variable types: 3 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  QObjective range [7e+02, 7e+02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e+00, 1e+03]
Found heuristic solution: objective 1421233.9650
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 263409 1.42123e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.6340864300

In [26]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculation
capital_cost = (capital_cost_production * num_facilities + 
                capital_cost_storage * num_storage_units + 
                capital_cost_transport * num_transport_units)

operating_cost = (operating_cost_production * num_facilities + 
                  operating_cost_storage * num_storage_units + 
                  operating_cost_transport * num_transport_units)

# Adding cost for extra production capacity
extra_capacity_cost = extra_capacity_per_facility * (capital_cost_production * num_facilities)

total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP calculation
total_GWP = (GWP_production * num_facilities + 
             GWP_storage * num_storage_units + 
             GWP_transport * num_transport_units)

# Risk calculation
total_risk = (risk_production * num_facilities + 
              risk_storage * num_storage_units + 
              risk_transport * num_transport_units)

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Add constraint to decide if extra facilities are needed
model.addConstr(num_facilities >= min_facilities, "Min facilities constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - (capital_cost_production * num_facilities_value + operating_cost_production * num_facilities_value)
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 5 nonzeros
Model fingerprint: 0x8cea05da
Model has 1 quadratic objective term
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  QObjective range [7e+02, 7e+02]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e+00, 1e+03]
Found heuristic solution: objective 1421233.9650
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 263409 1.42123e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.6340864300

In [39]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility ( crore ₹)
capital_cost_storage = 0.6        # per tonne ( crore ₹)
capital_cost_transport = 0.0075   # per tonne-km ( crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  #  crore ₹
operating_cost_storage = 0.00375   # ccrore ₹
operating_cost_transport = 0.00075 # crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

def display_results(num_facilities_value):
    # Fix the number of facilities for the current scenario
    num_facilities.setAttr("lb", num_facilities_value)
    num_facilities.setAttr("ub", num_facilities_value)
    
    # Solve the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Calculate total cost without loss
        actual_total_cost = (
            capital_cost_production * num_facilities_value
            + operating_cost_production * num_facilities_value
            + capital_cost_storage * num_storage_units.x
            + capital_cost_transport * num_transport_units.x
            + operating_cost_storage * num_storage_units.x
            + operating_cost_transport * num_transport_units.x
        )

        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {actual_total_cost}")  # Use actual total cost
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # Calculate and display loss if applicable
        if num_facilities_value == 2:
            loss_incurred = total_cost.getValue() - actual_total_cost
            print(f"Loss incurred for using 2 facilities: {loss_incurred}")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")


# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Number of Facilities: {num_facilities.x}")
else:
    print("Optimization was not successful.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0x541d9296
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 132478.21800
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 132478 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.324782180000e+05, best bound 1.324782180000e+05, gap 0.0000%

Results for 2 Facilities:
Optim

In [40]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (crore ₹)
capital_cost_storage = 0.6        # per tonne (crore ₹)
capital_cost_transport = 0.0075   # per tonne-km ( crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # crore ₹
operating_cost_storage = 0.00375   #  crore ₹
operating_cost_transport = 0.00075 # c crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day
facility_production_capacity = 500  # in tonnes per day

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Total cost calculation
total_cost = capital_cost + operating_cost

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Fix the number of facilities for the current scenario
    num_facilities.setAttr("lb", num_facilities_value)
    num_facilities.setAttr("ub", num_facilities_value)
    
    # Solve the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        actual_total_cost = (
            capital_cost_production * num_facilities_value
            + operating_cost_production * num_facilities_value
            + capital_cost_storage * num_storage_units.x
            + capital_cost_transport * num_transport_units.x
            + operating_cost_storage * num_storage_units.x
            + operating_cost_transport * num_transport_units.x
        )

        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {actual_total_cost}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        return {
            'facilities': num_facilities_value,
            'cost': actual_total_cost,
            'GWP': total_GWP.getValue(),
            'risk': total_risk.getValue()
        }
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")
        return None

# Solve for all facility scenarios and choose the best
results = []
for facilities in range(min_facilities, max_facilities + 1):
    result = display_results(facilities)
    if result:
        results.append(result)

# Find the optimal number of facilities based on cost, GWP, and risk
optimal_result = min(results, key=lambda x: (x['cost'], x['GWP'], x['risk']))

print("\nOptimal Solution:")
print(f"Optimal Number of Facilities: {optimal_result['facilities']}")
print(f"Total Cost (in crore ₹): {optimal_result['cost']}")
print(f"Total GWP (in 10^3 t CO2-equiv per day): {optimal_result['GWP']}")
print(f"Total Risk (unitless): {optimal_result['risk']}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x4d194be2
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 1547.7930000
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1547.79 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.547793000000e+03, best bound 1.547793000000e+03, gap 0.0000%

Results for 2 Facilities:
Opti

In [41]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility ( crore ₹)
capital_cost_storage = 0.6        # per tonne (crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # crore ₹
operating_cost_storage = 0.00375   # crore ₹
operating_cost_transport = 0.00075 #crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day
facility_production_capacity = 500  # in tonnes per day

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Total cost calculation
total_cost = capital_cost + operating_cost

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Fix the number of facilities for the current scenario
    num_facilities.setAttr("lb", num_facilities_value)
    num_facilities.setAttr("ub", num_facilities_value)
    
    # Solve the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Ensure the demand is met
        total_production = facility_production_capacity * num_facilities_value + extra_capacity_per_facility.x
        if total_production < demand:
            print(f"Results for {num_facilities_value} facilities are infeasible as they cannot meet the demand.")
            return None
        
        actual_total_cost = (
            capital_cost_production * num_facilities_value
            + operating_cost_production * num_facilities_value
            + capital_cost_storage * num_storage_units.x
            + capital_cost_transport * num_transport_units.x
            + operating_cost_storage * num_storage_units.x
            + operating_cost_transport * num_transport_units.x
        )

        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {actual_total_cost}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        return {
            'facilities': num_facilities_value,
            'cost': actual_total_cost,
            'GWP': total_GWP.getValue(),
            'risk': total_risk.getValue()
        }
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")
        return None

# Solve for all facility scenarios and choose the best
results = []
for facilities in range(min_facilities, max_facilities + 1):
    result = display_results(facilities)
    if result:
        results.append(result)

# Find the optimal number of facilities based on cost, GWP, and risk
optimal_result = min(
    (r for r in results if facility_production_capacity * r['facilities'] >= demand),
    key=lambda x: (x['cost'], x['GWP'], x['risk'])
)

print("\nOptimal Solution:")
print(f"Optimal Number of Facilities: {optimal_result['facilities']}")
print(f"Total Cost (in crore ₹): {optimal_result['cost']}")
print(f"Total GWP (in 10^3 t CO2-equiv per day): {optimal_result['GWP']}")
print(f"Total Risk (unitless): {optimal_result['risk']}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x4d194be2
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 1547.7930000
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1547.79 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.547793000000e+03, best bound 1.547793000000e+03, gap 0.0000%

Results for 2 Facilities:
Opti

In [42]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (crore ₹)
capital_cost_storage = 0.6        # per tonne (crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (crore ₹)

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  #  crore ₹
operating_cost_storage = 0.00375   # crore ₹
operating_cost_transport = 0.00075 #crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Facility setup cost (in crore ₹)
facility_setup_cost = 2000  # per facility

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Updated function to display results
def display_results(num_facilities_value):
    # Fix the number of facilities for the current scenario
    num_facilities.setAttr("lb", num_facilities_value)
    num_facilities.setAttr("ub", num_facilities_value)
    
    # Solve the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Ensure the demand is met
        total_production = facility_production_capacity * num_facilities_value + extra_capacity_per_facility.x
        if total_production < demand:
            print(f"Results for {num_facilities_value} facilities are infeasible as they cannot meet the demand.")
            return None
        
        # Total setup cost for facilities
        total_setup_cost = facility_setup_cost * num_facilities_value

        # Actual total cost including setup cost
        actual_total_cost = (
            total_setup_cost
            + capital_cost_production * num_facilities_value
            + operating_cost_production * num_facilities_value
            + capital_cost_storage * num_storage_units.x
            + capital_cost_transport * num_transport_units.x
            + operating_cost_storage * num_storage_units.x
            + operating_cost_transport * num_transport_units.x
        )

        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Setup Cost for Facilities (in crore ₹): {total_setup_cost}")
        print(f"Total Cost Including Setup (in crore ₹): {actual_total_cost}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        return {
            'facilities': num_facilities_value,
            'cost': actual_total_cost,
            'GWP': total_GWP.getValue(),
            'risk': total_risk.getValue()
        }
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")
        return None

# Check results for different numbers of facilities
results = []
for facilities in range(min_facilities, max_facilities + 1):
    result = display_results(facilities)
    if result:
        results.append(result)

# Find the optimal configuration based on feasibility and cost
if results:
    optimal_result = min(results, key=lambda x: x['cost'])
    print(f"\nOptimal Solution:")
    print(f"Optimal Number of Facilities: {optimal_result['facilities']}")
    print(f"Total Cost Including Setup (in crore ₹): {optimal_result['cost']}")
    print(f"Total GWP (in 10^3 t CO2-equiv per day): {optimal_result['GWP']}")
    print(f"Total Risk (unitless): {optimal_result['risk']}")
else:
    print("No feasible solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x746ebd33
Variable types: 3 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 4e+02]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 132478.21800
Presolve removed 3 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 132478 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.324782180000e+05, best bound 1.324782180000e+05, gap 0.0000%

Results for 2 Facilities:
Optim

In [45]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1369

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (to crore ₹)
capital_cost_storage = 0.6        # per tonne ( crore ₹)
capital_cost_transport = 0.0075   # per tonne-km ( crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost + facility_setup_cost * num_facilities

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If 2 facilities, calculate the loss
        if num_facilities_value > min_facilities:
            previous_facilities_cost = facility_setup_cost * (num_facilities_value - 1) + operating_cost_production * (num_facilities_value - 1)
            loss_incurred = total_cost.getValue() - previous_facilities_cost
            if loss_incurred < 0:
                print(f"Loss incurred by choosing {num_facilities_value} facilities is justified as it's less than setting up {num_facilities_value - 1} facilities.")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Display results for 2, 3, and 4 facilities
display_results(2)
display_results(3)
display_results(4)

# Display the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

for facilities in range(min_facilities, max_facilities + 1):
    num_facilities.setAttr("lb", facilities)
    num_facilities.setAttr("ub", facilities)
    model.optimize()

    if model.status == GRB.OPTIMAL:
        current_cost = total_cost.getValue()
        if current_cost < optimal_cost:
            optimal_cost = current_cost
            optimal_facilities = facilities

print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0xf503af6d
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective 207565.61800
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 207566 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.075656180000e+05, best bound 2.075656180000e+05, gap 0.0000%

Results for 2 Facilities:
Optim

In [54]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1510

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility ( crore ₹)
capital_cost_storage = 0.6        # per tonne ( crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  #  crore ₹
operating_cost_storage = 0.00375   #  crore ₹
operating_cost_transport = 0.00075 #  crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost + facility_setup_cost * num_facilities

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If more than the minimum facilities, calculate the loss
        if num_facilities_value > min_facilities:
            previous_facilities_cost = facility_setup_cost * (num_facilities_value - 1) + operating_cost_production * (num_facilities_value - 1)
            loss_incurred = total_cost.getValue() - previous_facilities_cost
            if loss_incurred < 0:
                print(f"Loss incurred by choosing {num_facilities_value} facilities is justified as it's less than setting up {num_facilities_value - 1} facilities.")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")


# Display results for a few facilities
display_results(2)
display_results(3)
display_results(4)

# Find the optimal number of facilities based on the lowest cost
optimal_facilities = None
optimal_cost = float('inf')

# Loop over the range of possible facilities
for facilities in range(min_facilities, max_facilities + 1):
    # Set the number of facilities for the current iteration
    num_facilities.setAttr("lb", facilities)
    num_facilities.setAttr("ub", facilities)
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        current_cost = total_cost.getValue()
        # Check if the current cost is lower than the previous optimal cost
        if current_cost < optimal_cost:
            optimal_cost = current_cost
            optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0xbc3b2d01
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective 186594.83500
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 186595 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.865948350000e+05, best bound 1.865948350000e+05, gap 0.0000%

Results for 2 Facilities:
Optim

In [56]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1050  # Change this value as required (e.g., for testing 1560, use demand = 1560)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost + facility_setup_cost * num_facilities

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Define the threshold for deciding to set up an additional facility
threshold_capacity_per_facility = 0.1  # 10% of the production capacity (can be adjusted based on business requirements)

# Function to calculate if the demand difference justifies setting up an additional facility
def should_add_facility(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    # If the demand is higher than the current capacity, calculate the difference
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, add another facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If more than the minimum facilities, calculate the loss
        if num_facilities_value > min_facilities:
            previous_facilities_cost = facility_setup_cost * (num_facilities_value - 1) + operating_cost_production * (num_facilities_value - 1)
            loss_incurred = total_cost.getValue() - previous_facilities_cost
            if loss_incurred < 0:
                print(f"Loss incurred by choosing {num_facilities_value} facilities is justified as it's less than setting up {num_facilities_value - 1} facilities.")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Display results for a few facilities
display_results(2)
display_results(3)
display_results(4)

# Find the optimal number of facilities based on the lowest cost
optimal_facilities = None
optimal_cost = float('inf')

# Loop over the range of possible facilities
for facilities in range(min_facilities, max_facilities + 1):
    # Check if adding a new facility is justified based on demand
    if should_add_facility(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_cost.getValue()
            # Check if the current cost is lower than the previous optimal cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0xbd36243b
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 23093.815000
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 23093.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.309381500000e+04, best bound 2.309381500000e+04, gap 0.0000%

Results for 2 Facilities:
Opti

In [57]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1050  # Change this value as required (e.g., for testing 1560, use demand = 1560)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost + facility_setup_cost * num_facilities

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Define the threshold for deciding to set up an additional facility
threshold_capacity_per_facility = 0.1  # 10% of the production capacity (can be adjusted based on business requirements)

# Function to calculate if the demand difference justifies setting up an additional facility
def should_add_facility(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    # If the demand is higher than the current capacity, calculate the difference
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, add another facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

        # If more than the minimum facilities, calculate the loss
        if num_facilities_value > min_facilities:
            previous_facilities_cost = facility_setup_cost * (num_facilities_value - 1) + operating_cost_production * (num_facilities_value - 1)
            loss_incurred = total_cost.getValue() - previous_facilities_cost
            if loss_incurred < 0:
                print(f"Loss incurred by choosing {num_facilities_value} facilities is justified as it's less than setting up {num_facilities_value - 1} facilities.")
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Display results for a few facilities
display_results(2)
display_results(3)
display_results(4)

# Find the optimal number of facilities based on the lowest cost
optimal_facilities = None
optimal_cost = float('inf')

# Loop over the range of possible facilities
for facilities in range(min_facilities, max_facilities + 1):
    # Ensure that the model always chooses at least the minimum number of facilities (e.g., 2)
    num_facilities.setAttr("lb", min_facilities)
    num_facilities.setAttr("ub", max_facilities)

    # Check if adding a new facility is justified based on demand
    if should_add_facility(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_cost.getValue()
            # Check if the current cost is lower than the previous optimal cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0xbd36243b
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 23093.815000
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 23093.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.309381500000e+04, best bound 2.309381500000e+04, gap 0.0000%

Results for 2 Facilities:
Opti

In [58]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
# Demand (in tonnes per day)
demand = 1050  # Change demand as per requirement, e.g., 1599, 1560, etc.

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")
total_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_cost")
total_GWP = model.addVar(vtype=GRB.CONTINUOUS, name="total_GWP")
total_risk = model.addVar(vtype=GRB.CONTINUOUS, name="total_risk")

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")
model.addConstr(facility_production_capacity * num_facilities >= demand, "Production capacity constraint")

# Cost and capacity related calculations
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units
total_cost_expr = capital_cost + operating_cost + facility_setup_cost * num_facilities
total_GWP_expr = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk_expr = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Total cost, GWP, and risk calculation
model.setObjective(total_cost_expr, GRB.MINIMIZE)

# Function to check if demand justifies adding a facility
def should_add_facility(current_demand, num_facilities, production_capacity_per_facility):
    current_capacity = num_facilities * production_capacity_per_facility
    if current_demand > current_capacity:
        return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Update the number of facilities in the model
    num_facilities.setAttr("lb", num_facilities_value)  # lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # upper bound
    
    # Re-optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

# Find the optimal number of facilities by iterating through possible values
optimal_facilities = None
optimal_cost = float('inf')

# Loop through all possible numbers of facilities within the range
for facilities in range(min_facilities, max_facilities + 1):
    # Set the bounds for number of facilities
    num_facilities.setAttr("lb", facilities)
    num_facilities.setAttr("ub", facilities)

    # Check if adding a new facility is justified based on demand
    if should_add_facility(demand, facilities, facility_production_capacity):
        # Optimize the model for this number of facilities
        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_cost.getValue()
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 3 nonzeros
Model fingerprint: 0x30b33f79
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 2 rows and 6 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

Optimal Number of Facilities: None


In [61]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
demand = 1560  # Change this value as required (e.g., for testing 1050, use demand = 1050)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Total cost calculation
total_cost = capital_cost + operating_cost + extra_capacity_cost + facility_setup_cost * num_facilities

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize cost)
model.setObjective(total_cost, GRB.MINIMIZE)

# Function to calculate cost of facilities and determine if it's justified to add a new facility
def compare_costs(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    
    # If the demand exceeds current capacity, consider adding another facility
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, it may justify an additional facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Cost (in crore ₹): {total_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        
    else:
        print(f"Optimization for {num_facilities_value} facilities was not successful.")

# Compare and choose the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

# Threshold for production capacity difference to add an additional facility (10% of the capacity)
threshold_capacity_per_facility = 0.1

# Loop over the possible number of facilities and compare costs
for facilities in range(min_facilities, max_facilities + 1):
    # Check if adding a new facility is justified based on demand
    if compare_costs(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_cost.getValue()
            # Compare the cost to previous best cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0x15249b1c
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [8e-03, 2e+03]
  Bounds range     [2e+00, 2e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective 204366.68500
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 204367 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.043666850000e+05, best bound 2.043666850000e+05, gap 0.0000%
Gurobi Optimizer version 12.0.0 

In [63]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
demand = 1499  # Change this value as required (e.g., for testing 1050, use demand = 1050)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Recovery time (in days)
recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Recovery cost per day (considering the time for recovery)
recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

# Total cost per day
total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

# Total yearly cost (for 365 days)
total_yearly_cost = total_daily_cost * 365

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Objective function (minimize yearly cost)
model.setObjective(total_yearly_cost, GRB.MINIMIZE)

# Function to calculate cost of facilities and determine if it's justified to add a new facility
def compare_costs(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    
    # If the demand exceeds current capacity, consider adding another facility
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, it may justify an additional facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        
    else:
        print(f"Optimization for {num_facilities_value} was not successful.")

# Compare and choose the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

# Threshold for production capacity difference to add an additional facility (10% of the capacity)
threshold_capacity_per_facility = 0.1

# Loop over the possible number of facilities and compare costs
for facilities in range(min_facilities, max_facilities + 1):
    # Check if adding a new facility is justified based on demand
    if compare_costs(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_yearly_cost.getValue()
            # Compare the cost to previous best cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 6 columns and 4 nonzeros
Model fingerprint: 0x9a11f7ad
Variable types: 5 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [3e-01, 1e+05]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 6.463263e+07
Presolve removed 3 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.46326e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.463262845750e+07, best bound 6.463262845750e+07, gap 0.0000%

Optimal Number of Faciliti

In [64]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
demand = 1499  # Change this value as required (e.g., for testing 1999, use demand = 1999)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Recovery time (in days)
recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Recovery cost per day (considering the time for recovery)
recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

# Total cost per day
total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

# Total yearly cost (for 365 days)
total_yearly_cost = total_daily_cost * 365

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Penalty for unmet demand
penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

# Calculate the unmet demand
model.addConstr(unmet_demand == GRB.max_(0, demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility)))

# Penalty cost for unmet demand
penalty_cost = penalty_cost_per_tonne * unmet_demand

# Add penalty cost to the total cost
total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

# Objective function (minimize yearly cost including penalty)
model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

# Function to calculate cost of facilities and determine if it's justified to add a new facility
def compare_costs(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    
    # If the demand exceeds current capacity, consider adding another facility
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, it may justify an additional facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        
    else:
        print(f"Optimization for {num_facilities_value} was not successful.")

# Compare and choose the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

# Threshold for production capacity difference to add an additional facility (10% of the capacity)
threshold_capacity_per_facility = 0.1

# Loop over the possible number of facilities and compare costs
for facilities in range(min_facilities, max_facilities + 1):
    # Check if adding a new facility is justified based on demand
    if compare_costs(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_yearly_cost_with_penalty.getValue()
            # Compare the cost to previous best cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


AttributeError: type object 'GRB' has no attribute 'max_'

In [66]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
demand = 1499  # Change this value as required (e.g., for testing 1999, use demand = 1999)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Recovery time (in days)
recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Recovery cost per day (considering the time for recovery)
recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

# Total cost per day
total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

# Total yearly cost (for 365 days)
total_yearly_cost = total_daily_cost * 365

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Variable to track unmet demand
unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

# Compute the unmet demand using the formula: unmet demand = max(0, demand - available capacity)
model.addConstr(unmet_demand >= demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility), "Unmet demand constraint")

# Penalty cost for unmet demand
penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
penalty_cost = penalty_cost_per_tonne * unmet_demand

# Add penalty cost to the total cost
total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

# Objective function (minimize yearly cost including penalty)
model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

# Function to calculate cost of facilities and determine if it's justified to add a new facility
def compare_costs(current_demand, num_facilities, production_capacity_per_facility, threshold_capacity_per_facility):
    # Calculate the total production capacity with current facilities
    current_capacity = num_facilities * production_capacity_per_facility
    
    # If the demand exceeds current capacity, consider adding another facility
    if current_demand > current_capacity:
        # If the demand difference is larger than the threshold, it may justify an additional facility
        if (current_demand - current_capacity) > threshold_capacity_per_facility * production_capacity_per_facility:
            return True
    return False

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        
    else:
        print(f"Optimization for {num_facilities_value} was not successful.")

# Compare and choose the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

# Threshold for production capacity difference to add an additional facility (10% of the capacity)
threshold_capacity_per_facility = 0.1

# Loop over the possible number of facilities and compare costs
for facilities in range(min_facilities, max_facilities + 1):
    # Check if adding a new facility is justified based on demand
    if compare_costs(demand, facilities, facility_production_capacity, threshold_capacity_per_facility):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", facilities)
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_yearly_cost_with_penalty.getValue()
            # Compare the cost to previous best cost
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 7 nonzeros
Model fingerprint: 0x6695394f
Variable types: 6 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-01, 1e+07]
  Bounds range     [2e+00, 2e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 6.447736e+09
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.44774e+09 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.447735838457e+09, best bound 6.447735838457e+09, gap 0.0000%

Optimal Number of Faciliti

In [70]:
from gurobipy import Model, GRB

# Input Data for Tamil Nadu
demand = 1369  # Change this value as required (e.g., for testing 1999, use demand = 1999)

# Facility costs (in crore ₹)
capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
facility_setup_cost = 2000        # cost to set up one facility in crore ₹

# Operating costs (in crore ₹ per day)
operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

# GWP (in 10^3 t CO2-equiv per day)
GWP_production = 135.27
GWP_storage = 70.33
GWP_transport = 0.002

# Risk (unitless scale)
risk_production = 580
risk_storage = 5155
risk_transport = 40

# Decision variables limits
min_facilities = 2
max_facilities = 4

# Assume each facility can produce a maximum of X tonnes per day (adjust based on real data)
facility_production_capacity = 500  # in tonnes per day (example capacity)

# Create the model
model = Model('Hydrogen Supply Chain Optimization')

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

# Cost calculation variables
total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

# Extra capacity per facility
extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

# Recovery time (in days)
recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

# Cost calculations for facilities and extra capacity
capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

# Adding cost for extra production capacity if needed
extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

# Recovery cost per day (considering the time for recovery)
recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

# Total cost per day
total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

# Total yearly cost (for 365 days)
total_yearly_cost = total_daily_cost * 365

# GWP and Risk calculations
total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

# Constraints
model.addConstr(num_storage_units >= demand, "Storage demand constraint")
model.addConstr(num_transport_units >= demand, "Transport demand constraint")

# Production capacity constraint: each facility should produce within its capacity
model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

# Variable to track unmet demand
unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

# Compute the unmet demand using the formula: unmet demand = max(0, demand - available capacity)
model.addConstr(unmet_demand >= demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility), "Unmet demand constraint")

# Penalty cost for unmet demand
penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
penalty_cost = penalty_cost_per_tonne * unmet_demand

# Add penalty cost to the total cost
total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

# Objective function (minimize yearly cost including penalty)
model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

# Function to calculate the minimum number of facilities required to meet demand
def calculate_min_facilities(demand, production_capacity_per_facility):
    # Calculate the minimum number of facilities required to meet the demand
    required_facilities = (demand + production_capacity_per_facility - 1) // production_capacity_per_facility  # Round up
    return required_facilities

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        
    else:
        print(f"Optimization for {num_facilities_value} was not successful.")

# Compare and choose the optimal number of facilities
optimal_facilities = None
optimal_cost = float('inf')

# Calculate the minimum number of facilities required
required_facilities = calculate_min_facilities(demand, facility_production_capacity)

# Loop over the possible number of facilities and compare costs
for facilities in range(min_facilities, max_facilities + 1):
    # Set the number of facilities for the current iteration
    num_facilities.setAttr("lb", max(facilities, required_facilities))  # Ensure the facilities are >= required_facilities
    num_facilities.setAttr("ub", facilities)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        current_cost = total_yearly_cost_with_penalty.getValue()
        # Compare the cost to previous best cost
        if current_cost < optimal_cost:
            optimal_cost = current_cost
            optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities: {optimal_facilities}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 7 nonzeros
Model fingerprint: 0xb877c4a5
Variable types: 6 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-01, 1e+05]
  Bounds range     [2e+00, 3e+00]
  RHS range        [1e+03, 1e+03]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, u

In [71]:
from gurobipy import Model, GRB

# Function to calculate the minimum number of facilities required to meet the demand
def calculate_min_facilities(demand, production_capacity_per_facility):
    # Calculate the minimum number of facilities required to meet the demand
    required_facilities = (demand + production_capacity_per_facility - 1) // production_capacity_per_facility  # Round up
    return required_facilities

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
    else:
        print(f"Optimization for {num_facilities_value} was not successful.")

# Main function to initialize and run the optimization model
def run_optimization():
    # Get user input for demand and facility parameters
    demand = int(input("Enter the demand (tonnes per day): "))
    min_facilities = int(input("Enter the minimum number of facilities: "))
    max_facilities = int(input("Enter the maximum number of facilities: "))
    production_capacity_per_facility = float(input("Enter the production capacity per facility (tonnes per day): "))

    # Facility costs (in crore ₹)
    capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
    capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
    capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
    facility_setup_cost = 2000        # cost to set up one facility in crore ₹

    # Operating costs (in crore ₹ per day)
    operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
    operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
    operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

    # GWP (in 10^3 t CO2-equiv per day)
    GWP_production = 135.27
    GWP_storage = 70.33
    GWP_transport = 0.002

    # Risk (unitless scale)
    risk_production = 580
    risk_storage = 5155
    risk_transport = 40

    # Create the model
    model = Model('Hydrogen Supply Chain Optimization')

    # Decision variables
    num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
    num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
    num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

    # Cost calculation variables
    total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
    total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

    # Extra capacity per facility
    extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

    # Recovery time (in days)
    recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

    # Cost calculations for facilities and extra capacity
    capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
    operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

    # Adding cost for extra production capacity if needed
    extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

    # Recovery cost per day (considering the time for recovery)
    recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

    # Total cost per day
    total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

    # Total yearly cost (for 365 days)
    total_yearly_cost = total_daily_cost * 365

    # GWP and Risk calculations
    total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
    total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

    # Constraints
    model.addConstr(num_storage_units >= demand, "Storage demand constraint")
    model.addConstr(num_transport_units >= demand, "Transport demand constraint")

    # Production capacity constraint: each facility should produce within its capacity
    model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

    # Variable to track unmet demand
    unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

    # Compute the unmet demand using the formula: unmet demand = max(0, demand - available capacity)
    model.addConstr(unmet_demand >= demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility), "Unmet demand constraint")

    # Penalty cost for unmet demand
    penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
    penalty_cost = penalty_cost_per_tonne * unmet_demand

    # Add penalty cost to the total cost
    total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

    # Objective function (minimize yearly cost including penalty)
    model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

    # Calculate the minimum number of facilities required
    required_facilities = calculate_min_facilities(demand, production_capacity_per_facility)

    # Loop over the possible number of facilities and compare costs
    for facilities in range(min_facilities, max_facilities + 1):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", max(facilities, required_facilities))  # Ensure the facilities are >= required_facilities
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_yearly_cost_with_penalty.getValue()
            display_results(facilities)

if __name__ == "__main__":
    run_optimization()


Enter the demand (tonnes per day):  1369
Enter the minimum number of facilities:  2
Enter the maximum number of facilities:  5
Enter the production capacity per facility (tonnes per day):  500


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 7 nonzeros
Model fingerprint: 0xb877c4a5
Variable types: 6 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-01, 1e+05]
  Bounds range     [2e+00, 3e+00]
  RHS range        [1e+03, 1e+03]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, u

In [72]:
from gurobipy import Model, GRB

# Function to calculate the minimum number of facilities required to meet the demand
def calculate_min_facilities(demand, production_capacity_per_facility):
    # Calculate the minimum number of facilities required to meet the demand
    required_facilities = (demand + production_capacity_per_facility - 1) // production_capacity_per_facility  # Round up
    return required_facilities

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")

# Main function to initialize and run the optimization model
def run_optimization():
    # Get user input for demand and facility parameters
    demand = int(input("Enter the demand (tonnes per day): "))
    min_facilities = int(input("Enter the minimum number of facilities: "))
    max_facilities = int(input("Enter the maximum number of facilities: "))
    production_capacity_per_facility = float(input("Enter the production capacity per facility (tonnes per day): "))

    # Facility costs (in crore ₹)
    capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
    capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
    capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
    facility_setup_cost = 2000        # cost to set up one facility in crore ₹

    # Operating costs (in crore ₹ per day)
    operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
    operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
    operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

    # GWP (in 10^3 t CO2-equiv per day)
    GWP_production = 135.27
    GWP_storage = 70.33
    GWP_transport = 0.002

    # Risk (unitless scale)
    risk_production = 580
    risk_storage = 5155
    risk_transport = 40

    # Create the model
    model = Model('Hydrogen Supply Chain Optimization')

    # Decision variables
    num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
    num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
    num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

    # Cost calculation variables
    total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
    total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

    # Extra capacity per facility
    extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

    # Recovery time (in days)
    recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

    # Cost calculations for facilities and extra capacity
    capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
    operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

    # Adding cost for extra production capacity if needed
    extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

    # Recovery cost per day (considering the time for recovery)
    recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

    # Total cost per day
    total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

    # Total yearly cost (for 365 days)
    total_yearly_cost = total_daily_cost * 365

    # GWP and Risk calculations
    total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
    total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

    # Constraints
    model.addConstr(num_storage_units >= demand, "Storage demand constraint")
    model.addConstr(num_transport_units >= demand, "Transport demand constraint")

    # Production capacity constraint: each facility should produce within its capacity
    model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

    # Variable to track unmet demand
    unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

    # Compute the unmet demand using the formula: unmet demand = max(0, demand - available capacity)
    model.addConstr(unmet_demand >= demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility), "Unmet demand constraint")

    # Penalty cost for unmet demand
    penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
    penalty_cost = penalty_cost_per_tonne * unmet_demand

    # Add penalty cost to the total cost
    total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

    # Objective function (minimize yearly cost including penalty)
    model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

    # Calculate the minimum number of facilities required
    required_facilities = calculate_min_facilities(demand, production_capacity_per_facility)

    optimal_facilities = None
    optimal_cost = float('inf')

    # Loop over the possible number of facilities and compare costs
    for facilities in range(min_facilities, max_facilities + 1):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", max(facilities, required_facilities))  # Ensure the facilities are >= required_facilities
        num_facilities.setAttr("ub", facilities)

        model.optimize()

        if model.status == GRB.OPTIMAL:
            current_cost = total_yearly_cost_with_penalty.getValue()

            # Update the optimal configuration if the current configuration is better
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

    # Display the optimal number of facilities
    print(f"\nOptimal Number of Facilities: {optimal_facilities}")
    display_results(optimal_facilities)

if __name__ == "__main__":
    run_optimization()


Enter the demand (tonnes per day):  1369
Enter the minimum number of facilities:  2
Enter the maximum number of facilities:  5
Enter the production capacity per facility (tonnes per day):  500


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 7 nonzeros
Model fingerprint: 0xb877c4a5
Variable types: 6 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-01, 1e+05]
  Bounds range     [2e+00, 3e+00]
  RHS range        [1e+03, 1e+03]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, u

In [73]:
from gurobipy import Model, GRB

# Function to calculate the minimum number of facilities required to meet the demand
def calculate_min_facilities(demand, production_capacity_per_facility):
    # Calculate the minimum number of facilities required to meet the demand
    required_facilities = (demand + production_capacity_per_facility - 1) // production_capacity_per_facility  # Round up
    return required_facilities

# Function to display results for a given number of facilities
def display_results(num_facilities_value):
    # Set the number of facilities to the current value
    num_facilities.setAttr("lb", num_facilities_value)  # update lower bound
    num_facilities.setAttr("ub", num_facilities_value)  # update upper bound
    
    model.optimize()

    if model.status == GRB.OPTIMAL:
        print(f"\nResults for {num_facilities_value} Facilities:")
        print(f"Optimal Number of Storage Units: {num_storage_units.x}")
        print(f"Optimal Number of Transport Units: {num_transport_units.x}")
        print(f"Extra Capacity per Facility (tonnes/day): {extra_capacity_per_facility.x}")
        print(f"Total Yearly Cost (in crore ₹): {total_yearly_cost_with_penalty.getValue()}")
        print(f"Total GWP (in 10^3 t CO2-equiv per day): {total_GWP.getValue()}")
        print(f"Total Risk (unitless): {total_risk.getValue()}")
        return total_yearly_cost_with_penalty.getValue()
    else:
        return None

# Main function to initialize and run the optimization model
def run_optimization():
    # Get user input for demand and facility parameters
    demand = int(input("Enter the demand (tonnes per day): "))
    min_facilities = int(input("Enter the minimum number of facilities: "))
    max_facilities = int(input("Enter the maximum number of facilities: "))
    production_capacity_per_facility = float(input("Enter the production capacity per facility (tonnes per day): "))

    # Facility costs (in crore ₹)
    capital_cost_production = 354.825  # per facility (converted from $47.31M to crore ₹)
    capital_cost_storage = 0.6        # per tonne (converted from $0.08M to crore ₹)
    capital_cost_transport = 0.0075   # per tonne-km (converted from $0.001M to crore ₹)
    facility_setup_cost = 2000        # cost to set up one facility in crore ₹

    # Operating costs (in crore ₹ per day)
    operating_cost_production = 0.1575  # converted from $0.021M to crore ₹
    operating_cost_storage = 0.00375   # converted from $0.0005M to crore ₹
    operating_cost_transport = 0.00075 # converted from $0.0001M to crore ₹

    # GWP (in 10^3 t CO2-equiv per day)
    GWP_production = 135.27
    GWP_storage = 70.33
    GWP_transport = 0.002

    # Risk (unitless scale)
    risk_production = 580
    risk_storage = 5155
    risk_transport = 40

    # Create the model
    model = Model('Hydrogen Supply Chain Optimization')

    # Decision variables
    num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
    num_storage_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_storage_units")
    num_transport_units = model.addVar(vtype=GRB.CONTINUOUS, name="num_transport_units")

    # Cost calculation variables
    total_capital_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_capital_cost")
    total_operating_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_operating_cost")

    # Extra capacity per facility
    extra_capacity_per_facility = model.addVar(vtype=GRB.CONTINUOUS, name="extra_capacity_per_facility", lb=0)

    # Recovery time (in days)
    recovery_time = 365  # e.g., recovery in 365 days, adjust as needed

    # Cost calculations for facilities and extra capacity
    capital_cost = capital_cost_production * num_facilities + capital_cost_storage * num_storage_units + capital_cost_transport * num_transport_units
    operating_cost = operating_cost_production * num_facilities + operating_cost_storage * num_storage_units + operating_cost_transport * num_transport_units

    # Adding cost for extra production capacity if needed
    extra_capacity_cost = extra_capacity_per_facility * capital_cost_production

    # Recovery cost per day (considering the time for recovery)
    recovery_cost_per_day = facility_setup_cost * num_facilities / recovery_time

    # Total cost per day
    total_daily_cost = operating_cost + recovery_cost_per_day + extra_capacity_cost

    # Total yearly cost (for 365 days)
    total_yearly_cost = total_daily_cost * 365

    # GWP and Risk calculations
    total_GWP = GWP_production * num_facilities + GWP_storage * num_storage_units + GWP_transport * num_transport_units
    total_risk = risk_production * num_facilities + risk_storage * num_storage_units + risk_transport * num_transport_units

    # Constraints
    model.addConstr(num_storage_units >= demand, "Storage demand constraint")
    model.addConstr(num_transport_units >= demand, "Transport demand constraint")

    # Production capacity constraint: each facility should produce within its capacity
    model.addConstr(facility_production_capacity * num_facilities + extra_capacity_per_facility >= demand, "Production capacity constraint")

    # Variable to track unmet demand
    unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

    # Compute the unmet demand using the formula: unmet demand = max(0, demand - available capacity)
    model.addConstr(unmet_demand >= demand - (facility_production_capacity * num_facilities + extra_capacity_per_facility), "Unmet demand constraint")

    # Penalty cost for unmet demand
    penalty_cost_per_tonne = 0.1  # Assume a penalty cost per tonne of unmet demand (in crore ₹)
    penalty_cost = penalty_cost_per_tonne * unmet_demand

    # Add penalty cost to the total cost
    total_yearly_cost_with_penalty = total_yearly_cost + penalty_cost

    # Objective function (minimize yearly cost including penalty)
    model.setObjective(total_yearly_cost_with_penalty, GRB.MINIMIZE)

    # Calculate the minimum number of facilities required
    required_facilities = calculate_min_facilities(demand, production_capacity_per_facility)

    optimal_facilities = None
    optimal_cost = float('inf')

    # Store all results to display later
    all_results = []

    # Loop over the possible number of facilities and compare costs
    for facilities in range(min_facilities, max_facilities + 1):
        # Set the number of facilities for the current iteration
        num_facilities.setAttr("lb", max(facilities, required_facilities))  # Ensure the facilities are >= required_facilities
        num_facilities.setAttr("ub", facilities)

        current_cost = display_results(facilities)
        
        if current_cost is not None:
            all_results.append((facilities, current_cost))
            # Update the optimal configuration if the current configuration is better
            if current_cost < optimal_cost:
                optimal_cost = current_cost
                optimal_facilities = facilities

    # Display all results
    print("\nComparison of all configurations:")
    for facilities, cost in all_results:
        print(f"{facilities} Facilities - Total Yearly Cost (with Penalty): {cost}")

    # Display the optimal number of facilities
    print(f"\nOptimal Number of Facilities: {optimal_facilities}")
    display_results(optimal_facilities)

if __name__ == "__main__":
    run_optimization()


Enter the demand (tonnes per day):  1369
Enter the minimum number of facilities:  1
Enter the maximum number of facilities:  4
Enter the production capacity per facility (tonnes per day):  500


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G7 CPU @ 1.20GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 7 nonzeros
Model fingerprint: 0x9cebda29
Variable types: 6 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e-01, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]

MIP start from previous solve produced solution with objective 1.12549e+08 (0.01s)
Loaded MIP start from previous solve with objective 1.12549e+08

Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1.12549e+08 

Optimal solution found (tolerance 1.00e-04)
Best 

In [74]:
# Import necessary libraries
from gurobipy import *
import numpy as np

# Assumed inputs from primary code or user
demand = float(input("Enter the demand for hydrogen production in tonnes per day: "))
min_facilities = int(input("Enter the minimum number of facilities: "))
max_facilities = int(input("Enter the maximum number of facilities: "))

# Renewable energy resources in Tamil Nadu (values should be realistic)
solar_energy_potential = 1000  # Solar energy potential in MW or MWh/day (adjust as needed)
wind_energy_potential = 800    # Wind energy potential in MW or MWh/day (adjust as needed)

# Cost of energy for electrolysis (₹ per kWh)
energy_cost_per_kWh = 0.05  # Example value for solar/wind energy (₹/kWh)

# Electrolyzer efficiency and cost per unit (how efficient is the conversion of energy to hydrogen)
electrolyzer_efficiency = 0.75  # 75% efficiency for hydrogen production
electrolyzer_cost_per_MW = 10   # Cost of electrolyzer per MW (₹ crore)

# Hydrogen production per MW of renewable energy (tonnes/day)
hydrogen_production_per_MW = 0.1  # Example value (adjust based on actual electrolyzer capacity)

# Land availability for setting up production facilities (in hectares)
land_per_facility = 5  # Example hectares per facility
land_available_in_region = 1000  # Total land available in hectares

# Government subsidies (₹ per unit)
subsidy_per_unit_of_renewable_energy = 0.02  # Subsidy per unit of renewable energy (₹/kWh)
subsidy_per_kg_of_hydrogen = 2.0  # Subsidy per kg of hydrogen produced (₹)

# Environmental Impact (Greenhouse Gas Emissions - GWP) per kg of hydrogen
GWP_per_kg_of_hydrogen = 0.01  # GWP reduction per kg of hydrogen produced (t CO2-equiv)

# Initialize the optimization model
model = Model("Green_Hydrogen_Production_Optimization")

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
hydrogen_production_capacity = model.addVar(vtype=GRB.CONTINUOUS, name="hydrogen_production_capacity")

# Calculate the capacity of hydrogen production per facility
model.addConstr(hydrogen_production_capacity == num_facilities * hydrogen_production_per_MW * electrolyzer_efficiency)

# Calculate total energy required
total_energy_required = model.addVar(vtype=GRB.CONTINUOUS, name="total_energy_required")
model.addConstr(total_energy_required == hydrogen_production_capacity / electrolyzer_efficiency)

# Cost parameters
energy_cost = model.addVar(vtype=GRB.CONTINUOUS, name="energy_cost")
model.addConstr(energy_cost == total_energy_required * energy_cost_per_kWh)

# Subsidies (for renewable energy and hydrogen production)
renewable_energy_subsidy = model.addVar(vtype=GRB.CONTINUOUS, name="renewable_energy_subsidy")
hydrogen_production_subsidy = model.addVar(vtype=GRB.CONTINUOUS, name="hydrogen_production_subsidy")

# Subsidy calculations
model.addConstr(renewable_energy_subsidy == total_energy_required * subsidy_per_unit_of_renewable_energy)
model.addConstr(hydrogen_production_subsidy == hydrogen_production_capacity * subsidy_per_kg_of_hydrogen)

# Total cost with subsidies
total_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_cost")
model.addConstr(total_cost == (num_facilities * electrolyzer_cost_per_MW) + energy_cost - renewable_energy_subsidy - hydrogen_production_subsidy)

# Environmental Impact (GWP reduction)
total_GWP_reduction = model.addVar(vtype=GRB.CONTINUOUS, name="total_GWP_reduction")
model.addConstr(total_GWP_reduction == hydrogen_production_capacity * GWP_per_kg_of_hydrogen)

# Land availability constraint
model.addConstr(num_facilities * land_per_facility <= land_available_in_region, "Land availability constraint")

# Unmet demand penalty
unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)
model.addConstr(unmet_demand == max(0, demand - hydrogen_production_capacity))

# Penalty cost for unmet demand
penalty_cost_per_tonne = 10  # Example penalty cost (₹ per tonne of unmet demand)
penalty_cost = model.addVar(vtype=GRB.CONTINUOUS, name="penalty_cost")
model.addConstr(penalty_cost == penalty_cost_per_tonne * unmet_demand)

# Total cost considering unmet demand penalty
total_cost_with_penalty = model.addVar(vtype=GRB.CONTINUOUS, name="total_cost_with_penalty")
model.addConstr(total_cost_with_penalty == total_cost + penalty_cost)

# Optimize the model
model.optimize()

# Display results for all facilities within the given range
if model.status == GRB.OPTIMAL:
    print("\nOptimized Results for Green Hydrogen Production:")
    print(f"Optimal Number of Facilities: {num_facilities.x}")
    print(f"Hydrogen Production Capacity (tonnes/day): {hydrogen_production_capacity.x}")
    print(f"Total Energy Cost (₹): {energy_cost.x}")
    print(f"Total Subsidies (₹): {renewable_energy_subsidy.x + hydrogen_production_subsidy.x}")
    print(f"Total Cost (₹): {total_cost.x}")
    print(f"Total GWP Reduction (t CO2-equiv per day): {total_GWP_reduction.x}")
    print(f"Total Penalty Cost for Unmet Demand (₹): {penalty_cost.x}")
    print(f"Total Cost with Penalty (₹): {total_cost_with_penalty.x}")

else:
    print("Optimization was not successful.")
    
# Function to compare and choose the best number of facilities
optimal_facilities = None
optimal_cost = float('inf')

for facilities in range(min_facilities, max_facilities + 1):
    num_facilities.setAttr("lb", facilities)
    num_facilities.setAttr("ub", facilities)
    model.optimize()

    if model.status == GRB.OPTIMAL:
        current_cost = total_cost_with_penalty.x
        if current_cost < optimal_cost:
            optimal_cost = current_cost
            optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities for Green Hydrogen Production: {optimal_facilities}")


Enter the demand for hydrogen production in tonnes per day:  1369
Enter the minimum number of facilities:  1
Enter the maximum number of facilities:  4


NotImplementedError: 

In [75]:
# Import necessary libraries
from gurobipy import *
import numpy as np

# Assumed inputs from primary code or user
demand = float(input("Enter the demand for hydrogen production in tonnes per day: "))
min_facilities = int(input("Enter the minimum number of facilities: "))
max_facilities = int(input("Enter the maximum number of facilities: "))

# Renewable energy resources in Tamil Nadu (values should be realistic)
solar_energy_potential = 1000  # Solar energy potential in MW or MWh/day (adjust as needed)
wind_energy_potential = 800    # Wind energy potential in MW or MWh/day (adjust as needed)

# Cost of energy for electrolysis (₹ per kWh)
energy_cost_per_kWh = 0.05  # Example value for solar/wind energy (₹/kWh)

# Electrolyzer efficiency and cost per unit (how efficient is the conversion of energy to hydrogen)
electrolyzer_efficiency = 0.75  # 75% efficiency for hydrogen production
electrolyzer_cost_per_MW = 10   # Cost of electrolyzer per MW (₹ crore)

# Hydrogen production per MW of renewable energy (tonnes/day)
hydrogen_production_per_MW = 0.1  # Example value (adjust based on actual electrolyzer capacity)

# Land availability for setting up production facilities (in hectares)
land_per_facility = 5  # Example hectares per facility
land_available_in_region = 1000  # Total land available in hectares

# Government subsidies (₹ per unit)
subsidy_per_unit_of_renewable_energy = 0.02  # Subsidy per unit of renewable energy (₹/kWh)
subsidy_per_kg_of_hydrogen = 2.0  # Subsidy per kg of hydrogen produced (₹)

# Environmental Impact (Greenhouse Gas Emissions - GWP) per kg of hydrogen
GWP_per_kg_of_hydrogen = 0.01  # GWP reduction per kg of hydrogen produced (t CO2-equiv)

# Initialize the optimization model
model = Model("Green_Hydrogen_Production_Optimization")

# Decision variables
num_facilities = model.addVar(vtype=GRB.INTEGER, name="num_facilities", lb=min_facilities, ub=max_facilities)
hydrogen_production_capacity = model.addVar(vtype=GRB.CONTINUOUS, name="hydrogen_production_capacity")

# Calculate the capacity of hydrogen production per facility
model.addConstr(hydrogen_production_capacity == num_facilities * hydrogen_production_per_MW * electrolyzer_efficiency)

# Calculate total energy required
total_energy_required = model.addVar(vtype=GRB.CONTINUOUS, name="total_energy_required")
model.addConstr(total_energy_required == hydrogen_production_capacity / electrolyzer_efficiency)

# Cost parameters
energy_cost = model.addVar(vtype=GRB.CONTINUOUS, name="energy_cost")
model.addConstr(energy_cost == total_energy_required * energy_cost_per_kWh)

# Subsidies (for renewable energy and hydrogen production)
renewable_energy_subsidy = model.addVar(vtype=GRB.CONTINUOUS, name="renewable_energy_subsidy")
hydrogen_production_subsidy = model.addVar(vtype=GRB.CONTINUOUS, name="hydrogen_production_subsidy")

# Subsidy calculations
model.addConstr(renewable_energy_subsidy == total_energy_required * subsidy_per_unit_of_renewable_energy)
model.addConstr(hydrogen_production_subsidy == hydrogen_production_capacity * subsidy_per_kg_of_hydrogen)

# Total cost with subsidies
total_cost = model.addVar(vtype=GRB.CONTINUOUS, name="total_cost")
model.addConstr(total_cost == (num_facilities * electrolyzer_cost_per_MW) + energy_cost - renewable_energy_subsidy - hydrogen_production_subsidy)

# Environmental Impact (GWP reduction)
total_GWP_reduction = model.addVar(vtype=GRB.CONTINUOUS, name="total_GWP_reduction")
model.addConstr(total_GWP_reduction == hydrogen_production_capacity * GWP_per_kg_of_hydrogen)

# Land availability constraint
model.addConstr(num_facilities * land_per_facility <= land_available_in_region, "Land availability constraint")

# Auxiliary variable for unmet demand (if hydrogen production is less than demand)
unmet_demand = model.addVar(vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

# Add constraints to ensure unmet demand is calculated correctly
model.addConstr(unmet_demand == max(0, demand - hydrogen_production_capacity))

# Penalty cost for unmet demand
penalty_cost_per_tonne = 10  # Example penalty cost (₹ per tonne of unmet demand)
penalty_cost = model.addVar(vtype=GRB.CONTINUOUS, name="penalty_cost")
model.addConstr(penalty_cost == penalty_cost_per_tonne * unmet_demand)

# Total cost considering unmet demand penalty
total_cost_with_penalty = model.addVar(vtype=GRB.CONTINUOUS, name="total_cost_with_penalty")
model.addConstr(total_cost_with_penalty == total_cost + penalty_cost)

# Optimize the model
model.optimize()

# Display results for all facilities within the given range
if model.status == GRB.OPTIMAL:
    print("\nOptimized Results for Green Hydrogen Production:")
    print(f"Optimal Number of Facilities: {num_facilities.x}")
    print(f"Hydrogen Production Capacity (tonnes/day): {hydrogen_production_capacity.x}")
    print(f"Total Energy Cost (₹): {energy_cost.x}")
    print(f"Total Subsidies (₹): {renewable_energy_subsidy.x + hydrogen_production_subsidy.x}")
    print(f"Total Cost (₹): {total_cost.x}")
    print(f"Total GWP Reduction (t CO2-equiv per day): {total_GWP_reduction.x}")
    print(f"Total Penalty Cost for Unmet Demand (₹): {penalty_cost.x}")
    print(f"Total Cost with Penalty (₹): {total_cost_with_penalty.x}")

else:
    print("Optimization was not successful.")
    
# Function to compare and choose the best number of facilities
optimal_facilities = None
optimal_cost = float('inf')

for facilities in range(min_facilities, max_facilities + 1):
    num_facilities.setAttr("lb", facilities)
    num_facilities.setAttr("ub", facilities)
    model.optimize()

    if model.status == GRB.OPTIMAL:
        current_cost = total_cost_with_penalty.x
        if current_cost < optimal_cost:
            optimal_cost = current_cost
            optimal_facilities = facilities

# Display the optimal number of facilities
print(f"\nOptimal Number of Facilities for Green Hydrogen Production: {optimal_facilities}")


Enter the demand for hydrogen production in tonnes per day:  1369
Enter the minimum number of facilities:  1
Enter the maximum number of facilities:  4


NotImplementedError: 